# Classification

TODO: riguardare i testi indicati con il tag TEMP<br>
TODO: rimuovere i testi con il tag REM

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix

In [ ]:
df_players_complete = pd.read_csv("./datasets/players.csv", index_col=0)

## Features choice

In [ ]:
# per il momento, gli stessi attributi utilizzati per il clustering REM
# problema: la feature mean_rank_points, da utilizzare come label, era già presente tra le features utilizzate per il clustering
df_players = df_players_complete[['max_tourney_revenue', 'lrpOnMxrp', 'matches_won_ratio', 'mean_rank_points']]

## Label computation

In [ ]:
df_players['is_high_ranked'] = np.digitize(df_players['mean_rank_points'], bins=[df_players['mean_rank_points'].median()])

In [ ]:
df_players[['mean_rank_points', 'is_high_ranked']]

In [ ]:
df_players = df_players.drop(columns=['mean_rank_points'])
df_players

## Preprocessing

In [ ]:
df_players_norm = pd.DataFrame(MinMaxScaler().fit_transform(df_players), columns=df_players.columns)
df_players_norm

X = df_players_norm.drop(columns=['is_high_ranked'])
Y = df_players_norm['is_high_ranked']
train_set, test_set, train_label, test_label = train_test_split(X, Y, stratify=Y, test_size=0.3, random_state=0)

## Classification

TODO: REM da questo punto in poi si tratta di copia e incolla adattati (dal progetto di Pasquali ecc.)

### Decision Tree

In [ ]:
from sklearn import tree

In [ ]:
dt = tree.DecisionTreeClassifier(criterion='gini', splitter='best', 
                                  max_depth=3, 
                                  min_samples_split=3, min_samples_leaf=8)
dt = dt.fit(train_set, train_label)

In [ ]:
from IPython.display import Image 
import pydotplus

classes = ['low ranked', 'high ranked']
dot_data = tree.export_graphviz(dt, out_file=None,
                         feature_names=list(train_set.columns),
                         class_names=classes,
                         filled=True, rounded=True)
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

#TODO: problema, cos'è il rettangolo nero?

In [ ]:


# predict and evaluate results on training and test set
train_pred_dt = dt.predict(train_set)
test_pred_dt = dt.predict(test_set)

print('Accuracy training set ', metrics.accuracy_score(train_label, train_pred_dt))
print('Accuracy test set ', metrics.accuracy_score(test_label, test_pred_dt))
print('Precision training set ', metrics.precision_score(train_label, train_pred_dt, average='weighted'))
print('Recall training set ', metrics.recall_score(train_label, train_pred_dt, average='weighted'))
print('F1 score trainig set ', metrics.f1_score(train_label, train_pred_dt, average='weighted'))
print('Support training set ', metrics.precision_recall_fscore_support(train_label, train_pred_dt))

In [ ]:
#TODO: plot_confusion_matrix deprecato, sostituirlo. Secondo problema: il kernel "muore" a questo punto
# plot confusion matrix
#plot_confusion_matrix(dt, test_set, test_label)
#plt.show() 

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# pretty printing of metrics computed on test set
def report_scores(test_label, test_pred):
    print(classification_report(test_label, 
                            test_pred, 
                            target_names=classes, zero_division=0)) 

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5, algorithm='ball_tree', metric='minkowski').fit(train_set, train_label)

test_pred_knn = knn.predict(test_set)

report_scores(test_label,test_pred_knn)